In [14]:
import os
import sys
from omegaconf import DictConfig, ListConfig

import hydra
from hydra import utils
import mlflow
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split


In [4]:
import mlflow
mlflow.set_tracking_uri("http://at-compute015:5000/")

In [8]:
def log_params_from_omegaconf_dict(params):
    for param_name, element in params.items():
        _explore_recursive(param_name, element)

def _explore_recursive(parent_name, element):
    if isinstance(element, DictConfig):
        for k, v in element.items():
            if isinstance(v, DictConfig) or isinstance(v, ListConfig):
                _explore_recursive(f'{parent_name}.{k}', v)
            else:
                mlflow.log_param(f'{parent_name}.{k}', v)
    elif isinstance(element, ListConfig):
        for i, v in enumerate(element):
            mlflow.log_param(f'{parent_name}.{i}', v)    

In [16]:
import mlflow

# Create nested runs
experiment_id = mlflow.create_experiment("experiment4")
with mlflow.start_run(
    run_name="PARENT_RUN",
    experiment_id=experiment_id,
    tags={"version": "v1", "priority": "P1"},
    description="parent",):
    
    mlflow.log_param("lr", 0.001)
    # Your ml code
    ...
    mlflow.log_metric("val/loss", 0.1)
    log_params_from_omegaconf_dict(cfg)


2024/10/25 23:06:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run PARENT_RUN at: http://at-compute015:5000/#/experiments/604526792052960403/runs/031660f3c7bd4713ae985a72ce26fae8.
2024/10/25 23:06:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://at-compute015:5000/#/experiments/604526792052960403.


In [ ]:

# Search all child runs with a parent id
query = f"tags.mlflow.parentRunId = '{parent_run.info.run_id}'"
results = mlflow.search_runs(experiment_ids=[experiment_id], filter_string=query)
print("child runs:")
print(results[["run_id", "params.child", "tags.mlflow.runName"]])

# Create a nested run under the existing parent run
with mlflow.start_run(
    run_name="NEW_CHILD_RUN",
    experiment_id=experiment_id,
    description="new child",
    parent_run_id=parent_run.info.run_id,
) as child_run:
    mlflow.log_param("new-child", "yes")

In [9]:
from experanto.configs import DEFAULT_CONFIG as cfg